In [1]:
library(fastDummies)
library(cplm)
library(tweedie)
library(statmod)
library(HDtweedie)
library(TDboost)
library(mgcv)
library(MASS)
library(glmnet)

Loading required package: coda

Loading required package: Matrix

Loading required package: splines

Loading required package: lattice

Loaded TDboost 1.2


Loading required package: nlme


Attaching package: 'nlme'


The following objects are masked from 'package:cplm':

    fixef, ranef, VarCorr


This is mgcv 1.8-38. For overview type 'help("mgcv-package")'.

Loaded glmnet 4.1-2



In [2]:
source("S_U.R")

Without standardization, it will have fitting problem.

In [4]:
x1<-rnorm(10000,0,1)
x2<-rnorm(10000,0,1)
x3<-rnorm(10000,0,1)
X<-as.data.frame(cbind(x1,x2,x3))
b_t<-runif(5,-2,1)
design_x <- model.matrix(~.+x1*x2,data=X)
mu<-exp(design_x%*%b_t)
summary(mu)

       V1           
 Min.   :  0.00102  
 1st Qu.:  0.27046  
 Median :  0.57969  
 Mean   :  0.84027  
 3rd Qu.:  1.05024  
 Max.   :144.72393  

In [5]:
y_sim<- rTweedie(mu,p=1.5,phi=10)
length(which(y_sim==0))

[1] 8515

In [6]:
dd<-as.data.frame(cbind(design_x[,2:4],y_sim))
sam <- sample(rep(1:2,len=10000))
sim_train<-dd[sam==1,]
sim_test<-dd[sam==2,]
sim_train<-as.data.frame(sim_train)
sim_test<-as.data.frame(sim_test)
colnames(sim_train)[4]='y'
colnames(sim_test)[4]='y'

In [7]:
fit1<-glm(y~.+x1*x2,data=sim_train,family=tweedie(link.power=0,var.power=1.5))
summary(fit1)
mean(residuals(fit1)^2)
b_t


Call:
glm(formula = y ~ . + x1 * x2, family = tweedie(link.power = 0, 
    var.power = 1.5), data = sim_train)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-7.261  -1.965  -1.651  -1.202   9.987  

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.65850    0.05525 -11.920   <2e-16 ***
x1           0.82700    0.04740  17.448   <2e-16 ***
x2           0.49318    0.05161   9.556   <2e-16 ***
x3          -0.08082    0.04856  -1.664   0.0961 .  
x1:x2       -0.45199    0.04226 -10.696   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Tweedie family taken to be 9.821138)

    Null deviance: 24586  on 4999  degrees of freedom
Residual deviance: 20713  on 4995  degrees of freedom
AIC: NA

Number of Fisher Scoring iterations: 5


[1] 4.142596

[1] -0.71285417  0.87784393  0.49481697 -0.03479301 -0.41090671

Change initialization

In [8]:
fit_0<-glm(y~.^2,data=sim_train,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500))
fit_0$coefficients

In [185]:
# #Initial: For example, we can use the least square estimates or the simple regression estimates by regressing the response y on each of the terms.

# fit_b0<-glm(y~1,data=sim_train,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500))
# b0_0 <- fit_b0$coefficients

# # Main effects

# fit_b1<-glm(y~x1-1,data=sim_train,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500))
# b1_0 <- fit_b1$coefficients

# fit_b2<-glm(y~x2-1,data=sim_train,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500))
# b2_0 <- fit_b2$coefficients

# fit_b3<-glm(y~x3-1,data=sim_train,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500))
# b3_0 <- fit_b3$coefficients
# #Interaction: when update r12, use y~b1b2x1x2 with no intercept.
# effect_12<-b1_0*b2_0*sim_train$x1*sim_train$x2
# fit_r12<-glm(sim_train$y~effect_12-1,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500))
# r12_0<-fit_r12$coefficients

# effect_13<-b1_0*b3_0*sim_train$x1*sim_train$x3
# fit_r13<-glm(sim_train$y~effect_13-1,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500))
# r13_0<-fit_r13$coefficients

# effect_23<-b1_0*b2_0*sim_train$x2*sim_train$x3
# fit_r23<-glm(sim_train$y~effect_23-1,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500))
# r23_0<-fit_r23$coefficients

In [10]:
b0_0<-fit_0$coefficients[1]
b1_0<-fit_0$coefficients[2]
b2_0<-fit_0$coefficients[3]
b3_0<-fit_0$coefficients[4]
r12_0<-fit_0$coefficients[5]/(b1_0*b2_0)
r13_0<-fit_0$coefficients[6]/(b1_0*b3_0)
r23_0<-fit_0$coefficients[7]/(b2_0*b3_0)

In [11]:
effect_12<-b1_0*b2_0*sim_train$x1*sim_train$x2
effect_13<-b1_0*b3_0*sim_train$x1*sim_train$x3
effect_23<-b1_0*b2_0*sim_train$x2*sim_train$x3

In [ ]:
X1<-sim_train$x1
X2<-sim_train$x2
X3<-sim_train$x3
X<-as.matrix(cbind(X1,X2,X3))

Lambda choice

In [12]:
mysd <- function(z) sqrt(sum((z-mean(z))^2)/length(z))
sx <- scale(X, scale = apply(X, 2, mysd))
sx <- as.matrix(sx)
sy=sim_train$y

In [13]:
lambda_max <- max(abs(colSums(sx*sy)))/5000

In [14]:
epsilon <- .0001
K <- 10
lambdapath <- round(exp(seq(log(lambda_max), log(lambda_max*epsilon), 
                            length.out = K)), digits = 10)
lambdapath

[1] 0.1157734357 0.0416068155 0.0149527142 0.0053737269 0.0019312173
 [6] 0.0006940435 0.0002494263 0.0000896392 0.0000322146 0.0000115773

In [37]:
r_update<-function(beta,power,y,X,lambda){
    offset=as.matrix(cbind(1,X))%*%beta
    beta_x<-beta[-1]
    n<-length(beta_x)
    design_matrix<-c()
    index<-combn(c(1:n),m=2)
    for (i in 1:n){
        beta_tmp<-beta_x[index[1,i]]*beta_x[index[2,i]]
        design_matrix<-cbind(design_matrix,beta_tmp*X[,index[1,i]]*X[,index[2,i]]  )
    }
    design_matrix<-as.matrix(design_matrix)
    r_fit<-glmnet(x=design_matrix,y=y,family=tweedie(link.power=0,var.power=power),offset=offset,intercept=FALSE,lambda=lambda,control=list(maxit=500))
    return(coef(r_fit)[-1,1])
}

Here divergence or not converge does not affect beta when adding more iterations

In [350]:
#Update b1. To note design matrix is (0, x1)
offset_b1<-b2_0*X2+b3_0*X3+b2_0*b3_0*X2*X3+b0_0
b1_x1<-X1+r_update_1[1]*b2_0*X1*X2+r_update_1[2]*b3_0*X1*X3
b1_matrix <- (as.matrix(cbind(0,b1_x1)))
b1_1_fit<-glmnet(x=b1_matrix,y=sim_train$y,family=tweedie(link.power=0,var.power=1.5)
                    ,offset=offset_b1,intercept=FALSE,lambda=0.08,control=list(maxit=5000))
b1_update<-coef(b1_1_fit)[,1]
b1_update_1<-b1_update[3]
b1_update_1

Warning message:
"Infinite objective function!"
Warning message:
"step size truncated due to divergence"


b1_x1 
-1.941848

In [351]:
#Update b2
offset_b2<-b1_update_1*X1+b3_0*X3+b1_update_1*b3_0*X1*X3+b0_0
b2_x1<-X2+r_update_1[1]*b1_update_1*X1*X2+r_update_1[3]*b3_0*X2*X3
b2_matrix <- as.matrix(cbind(0,b2_x1))
b2_1_fit<-glmnet(x=b2_matrix,y=sim_train$y,family=tweedie(link.power=0,var.power=1.5)
                    ,offset=offset_b2,intercept=FALSE,lambda=0.08)
b2_update<-coef(b2_1_fit,s='lambda.min')[,1]
b2_update_1<-b2_update[3]

In [340]:
#Update b3
offset_b3<-b1_update_1*X1+b2_update_1*X2+b1_update_1*b2_update_1*X1*X2+b0_0
b3_x1<-X3+r_update_1[2]*b1_update_1*X1*X3+r_update_1[3]*b2_update_1*X2*X3
b3_matrix <- (as.matrix(cbind(0,b3_x1)))
b3_1_fit<-glmnet(x=b3_matrix,y=sim_train$y,family=tweedie(link.power=0,var.power=1.5),offset=offset_b3,control=list(maxit=5000)
                    ,lambda=0.08,intercept=FALSE)
b3_update<-coef(b3_1_fit,s='lambda.min')[,1]
b3_update_1<-b3_update[3]

In [341]:
#Update b0
offset_b0<-b1_update_1*X1+b2_update_1*X2+b3_update_1*X3+b1_update_1*b2_update_1*X1*X2+b2_update_1*b3_update_1*X2*X3+b1_update_1*b3_update_1*X1*X3   
b0_1_fit<-glm(sim_train$y~1,offset=offset_b0,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500))
b0_update_1<-b0_1_fit$coefficients

In [342]:
b0_update_1
b1_update_1
b2_update_1
b3_update_1
r_update_1

(Intercept) 
  0.6026401

b1_x1 
-1.941848

b2_x1 
-0.4238372

b3_x1 
0.07912313

effect_12 effect_13 effect_23 
0.7763744 0.0000000 0.0000000

In [266]:
b_update_1<-c(b1_update_1,b2_update_1,b3_update_1)

In [358]:
Q_r<-function(y,mu,power,lamb_r,lamb_b,r,b){
    den<-tweedie.dev(y=y,mu=mu,power=power)
    Q<-sum(den,na.rm=T)+sum(abs(r))*lamb_r+sum(abs(b[-1]))*lamb_b
    return(Q)
}

In [17]:
options(warn=-1)
r_0_t<-c(r12_0,r13_0,r23_0)
beta_0_t<-c(b0_0,b1_0, b2_0, b3_0)

In [18]:
r_lamb<-lambdapath
beta_lamb<-lambdapath
n_lamb<-length(beta_lamb)
power=1.5
final_result=matrix(nrow=n_lamb,ncol=n_lamb)

In [81]:
for (i in 1:n_lamb){
    for (j in 1:n_lamb){
        coef_result<-shim_lasso_update_2(sim_train[,1:3],sim_train$y,power,lamb_r=r_lamb[i],lamb_b=beta_lamb[j],r_0_t,beta_0_t)
        fitted<-ts_fitted(sim_train[,1:3],coef_result$'beta',coef_result$'r')
        final_result[i,j]=mean(tweedie.dev(sim_train$y,fitted,1.5))
}
    }

In [83]:
which(final_result==min(final_result,na.rm=T), arr.ind = TRUE)

row,col
7,3


In [86]:
min(final_result,na.rm=T)
mean(residuals(fit1)^2)

[1] 4.161979

[1] 4.142596

In [76]:
coef_result<-shim_lasso_update_2(sim_train[,1:3],sim_train$y,power,lamb_r=r_lamb[7],lamb_b=beta_lamb[3],r_0_t,beta_0_t)

In [77]:
coef_result$beta
coef_result$r

(Intercept)                                     
-0.49068942  0.81578551  0.48157136 -0.05987724

V1         V2         V3 
-1.1066907  0.8405046  0.1945627

In [88]:
r_lamb[7]
beta_lamb[3]

[1] 0.0002494263

[1] 0.01495271

In [90]:
coef_result$beta[2]*coef_result$beta[3]*coef_result$r[1]
coef_result$beta[2]*coef_result$beta[4]*coef_result$r[2]
coef_result$beta[3]*coef_result$beta[4]*coef_result$r[3]
coef_result$beta

-0.4539956

-0.0471088

-0.006808385

(Intercept)                                     
-0.49556912  0.82822957  0.49506392 -0.06725156

In [84]:
b_t

[1] -0.71285417  0.87784393  0.49481697 -0.03479301 -0.41090671